# A Transfer Learning and Optimized CNN Based Intrusion Detection System for Internet of Vehicles 
This is the code for the paper entitled "**A Transfer Learning and Optimized CNN Based Intrusion Detection System for Internet of Vehicles**" accepted in IEEE International Conference on Communications (IEEE ICC).  
Authors: Li Yang (lyang339@uwo.ca) and Abdallah Shami (Abdallah.Shami@uwo.ca)  
Organization: The Optimized Computing and Communications (OC2) Lab, ECE Department, Western University

**Notebook 2: CNN Model Development**  
Aims:  
&nbsp; 1): Generate training and test images  
&nbsp; 2): Construct CNN models (a CNN model by own, Xception, VGG16, VGG19, Resnet, Inception, InceptionResnet)  
&nbsp; 3): Tune the hyperparameters of CNN models (hyperparameter optimization)  

## Import libraries

In [1]:
import os
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,GlobalAveragePooling2D,Input,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.applications.xception import  Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import  ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.mobilenet import MobileNet
import tensorflow.keras.callbacks as kcallbacks
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import math
import random
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from PIL import Image
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

## Generate Training and Test Images

In [2]:
#generate training and test images
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)
BATCHSIZE=128	#could try 128 or 32

#Normalization
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train_224/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        './test_224/',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

Found 2076 images belonging to 5 classes.
Found 518 images belonging to 5 classes.


### Define the image plotting functions

In [3]:
import pandas as pd
import time
output_df = pd.DataFrame(columns=['Loss','Accuracy',  'Val-Loss', 'Val-Accuracy', 'Time'])
output_index = list()
timelist=list()

In [4]:
#plot the figures
class LossHistory(keras.callbacks.Callback):
    def __init__(self, model_name):
        self.model_name=model_name
    def on_train_begin(self, logs={}):
        
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}
    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('accuracy'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_accuracy'))
    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('accuracy'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_accuracy'))
    
    #把数据存入df中
    def on_train_end(self, logs={}):
        global output_df
        global output_index
        idx=0
        maxValue=0
       
        for i in range(len(self.val_acc['epoch'])):
            if self.val_acc['epoch'][i]>maxValue :
                maxValue=self.val_acc['epoch'][i]
                idx=i
        
        result_dict={
            'Loss': self.losses['epoch'][idx],
            'Accuracy':self.accuracy['epoch'][idx],
            'Val-Loss':self.val_loss['epoch'][idx],
            'Val-Accuracy':self.val_acc['epoch'][idx],
        }
        output_df=output_df.append(result_dict,ignore_index=True)
        output_index.append(self.model_name)    
    def loss_plot(self, loss_type): 
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # acc
            plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
            # loss
            plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()

In [5]:
history_CNN= LossHistory("CNN")
history_Xception= LossHistory("Xception")
history_Inception= LossHistory("Inception")
history_VGG19= LossHistory("VGG19")
history_Resnet= LossHistory("Resnet")
history_InceptionResnet= LossHistory("InceptionResnet")
history_VGG16= LossHistory("VGG16")


# Construct CNN models

### Model 1: a CNN model by own (baseline)

In [7]:
def cnn_by_own(input_shape,num_class,epochs,savepath='./model_own.h5'):
    model = Sequential()
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=input_shape,padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(num_class,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    #train model
    earlyStopping=kcallbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, mode='auto')
    saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
    hist=model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        callbacks=[earlyStopping,saveBestModel,history_CNN],
    )
    

In [8]:

start_time=time.time()
cnn_by_own(input_shape=INPUT_SIZE,num_class=5,epochs=20)
end_time=time.time()
timelist.append(end_time-start_time)


c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 15s 497ms/step - loss: 1.3046 - accuracy: 0.6373 - val_loss: 1.1782 - val_accuracy: 0.6409

Epoch 00001: val_accuracy improved from -inf to 0.64093, saving model to .\model_own.h5
Epoch 2/20
17/17 [==============================] - 7s 376ms/step - loss: 1.0370 - accuracy: 0.6705 - val_loss: 0.9569 - val_accuracy: 0.6409

Epoch 00002: val_accuracy did not improve from 0.64093
Epoch 3/20
17/17 [==============================] - 7s 375ms/step - loss: 0.7965 - accuracy: 0.7187 - val_loss: 0.6659 - val_accuracy: 0.7452

Epoch 00003: val_accuracy improved from 0.64093 to 0.74517, saving model to .\model_own.h5
Epoch 4/20
17/17 [==============================] - 7s 374ms/step - loss: 0.5303 - accuracy: 0.8362 - val_loss: 0.3068 - val_accuracy: 0.8707

Epoch 00004: val_accuracy improved from 0.74517 to 0.87066, saving model to .\model_own.h5
Epoch 5/20
17/17 [==============================] - 7s 374ms/step - loss: 0.3170 - accuracy: 0.8849 - 

C:\Users\FOCUS\AppData\Local\Temp\ipykernel_22608\4074440202.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [10]:
#output_df
timelist

[-41.485483169555664]

Validation accuracy of a CNN by own: 99.884%

### Model 2: Xception

In [9]:
def xception( num_class, epochs,savepath='./xception.h5',history=history_Xception,input_shape=INPUT_SIZE):
    model_fine_tune = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:131]:		#could be tuned to be 50, 100, or 131
        layer.trainable = False
    for layer in model_fine_tune.layers[131:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='xception')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=3, verbose=1, mode='auto')	#patience could be tuned by 2 and 3
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
    )


In [10]:
#default only 50, tf36cnn 99
start_time=time.time()
xception(num_class=5,epochs=20)
end_time=time.time()
timelist.append(end_time-start_time)

# Insufficient Video Memory 显存不足

c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
17/17 [==============================] - 8s 322ms/step - loss: 0.6212 - accuracy: 0.7789 - val_loss: 0.1624 - val_accuracy: 0.9363

Epoch 00001: val_accuracy improved from -inf to 0.93629, saving model to .\xception.h5


c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
17/17 [==============================] - 4s 261ms/step - loss: 0.1214 - accuracy: 0.9639 - val_loss: 0.0520 - val_accuracy: 1.0000

Epoch 00002: val_accuracy improved from 0.93629 to 1.00000, saving model to .\xception.h5
Epoch 3/20
17/17 [==============================] - 5s 282ms/step - loss: 0.0587 - accuracy: 0.9855 - val_loss: 0.0334 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/20
17/17 [==============================] - 5s 262ms/step - loss: 0.0360 - accuracy: 0.9947 - val_loss: 0.0205 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/20
17/17 [==============================] - 6s 385ms/step - loss: 0.0265 - accuracy: 0.9971 - val_loss: 0.0146 - val_accuracy: 1.0000

Epoch 00005: val_accuracy did not improve from 1.00000
Epoch 00005: early stopping


C:\Users\FOCUS\AppData\Local\Temp\ipykernel_22608\4074440202.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [18]:
timelist
output_df

,Loss,Accuracy,Val-Loss,Val-Accuracy,Time
0,0.350683,0.872832,0.218641,0.907336,NaN
1,0.620470,0.809249,0.411984,0.870656,NaN
2,0.057802,0.988439,0.029836,1.000000,NaN
3,0.040710,0.994701,0.022890,1.000000,NaN


Validation accuracy of Xception: 100.0%

### Model 3: VGG16

In [11]:
def vgg16( num_class, epochs,savepath='./VGG16.h5',history=history_VGG16,input_shape=INPUT_SIZE):
    model_fine_tune = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:15]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[15:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output) #GlobalAveragePooling2D layer to convert the features to a single 1280-element vector per image
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[earlyStopping, saveBestModel, history],
    )


In [12]:
start_time=time.time()
vgg16(num_class=5,epochs=20)	#tf36cnn
end_time=time.time()
timelist.append(end_time-start_time)
# Insufficient Video Memory 显存不足

c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 11s 556ms/step - loss: 0.9956 - accuracy: 0.6267 - val_loss: 0.4159 - val_accuracy: 0.8571

Epoch 00001: val_accuracy improved from -inf to 0.85714, saving model to .\VGG16.h5
Epoch 2/20
17/17 [==============================] - 8s 440ms/step - loss: 0.3562 - accuracy: 0.8637 - val_loss: 0.2961 - val_accuracy: 0.8861

Epoch 00002: val_accuracy improved from 0.85714 to 0.88610, saving model to .\VGG16.h5
Epoch 3/20
17/17 [==============================] - 9s 538ms/step - loss: 0.2018 - accuracy: 0.9138 - val_loss: 0.1130 - val_accuracy: 0.9247

Epoch 00003: val_accuracy improved from 0.88610 to 0.92471, saving model to .\VGG16.h5
Epoch 4/20
17/17 [==============================] - 9s 509ms/step - loss: 0.0813 - accuracy: 0.9658 - val_loss: 0.0207 - val_accuracy: 1.0000

Epoch 00004: val_accuracy improved from 0.92471 to 1.00000, saving model to .\VGG16.h5
Epoch 5/20
17/17 [==============================] - 9s 485ms/step - loss: 0.1169 -

C:\Users\FOCUS\AppData\Local\Temp\ipykernel_22608\4074440202.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of VGG16: 100.0%

### Model 4: VGG19

In [13]:
def vgg19( num_class, epochs,savepath='./VGG19.h5',history=history_VGG19,input_shape=INPUT_SIZE):
    model_fine_tune = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:19]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[19:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[earlyStopping, saveBestModel, history],
    )


In [14]:
start_time=time.time()
vgg19(num_class=5,epochs=20)	#binary classificaiton

end_time=time.time()
timelist.append(end_time-start_time)

c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 12s 599ms/step - loss: 0.7404 - accuracy: 0.7163 - val_loss: 0.2279 - val_accuracy: 0.9286

Epoch 00001: val_accuracy improved from -inf to 0.92857, saving model to .\VGG19.h5
Epoch 2/20
17/17 [==============================] - 7s 397ms/step - loss: 0.1427 - accuracy: 0.9494 - val_loss: 0.0096 - val_accuracy: 1.0000

Epoch 00002: val_accuracy improved from 0.92857 to 1.00000, saving model to .\VGG19.h5
Epoch 3/20
17/17 [==============================] - 5s 265ms/step - loss: 0.0235 - accuracy: 0.9899 - val_loss: 0.0033 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/20
17/17 [==============================] - 5s 269ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 1.8429e-04 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 00004: early stopping


C:\Users\FOCUS\AppData\Local\Temp\ipykernel_22608\4074440202.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of VGG19: 100.0%

### Model 5: ResNet

In [15]:
def resnet( num_class, epochs,savepath='./resnet.h5',history=history_Resnet,input_shape=INPUT_SIZE):
    model_fine_tune = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:120]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[120:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
    )

In [16]:
start_time=time.time()
resnet(num_class=5,epochs=20)	#binary classificaiton
end_time=time.time()
timelist.append(end_time-start_time)

c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 9s 355ms/step - loss: 0.7253 - accuracy: 0.7582 - val_loss: 25.0492 - val_accuracy: 0.6409

Epoch 00001: val_accuracy improved from -inf to 0.64093, saving model to .\resnet.h5


c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
17/17 [==============================] - 5s 268ms/step - loss: 0.2117 - accuracy: 0.9128 - val_loss: 37.0571 - val_accuracy: 0.6409

Epoch 00002: val_accuracy did not improve from 0.64093
Epoch 3/20
17/17 [==============================] - 5s 274ms/step - loss: 0.1279 - accuracy: 0.9509 - val_loss: 12.1715 - val_accuracy: 0.7452

Epoch 00003: val_accuracy improved from 0.64093 to 0.74517, saving model to .\resnet.h5
Epoch 4/20
17/17 [==============================] - 5s 261ms/step - loss: 0.0720 - accuracy: 0.9750 - val_loss: 2.4784 - val_accuracy: 0.8707

Epoch 00004: val_accuracy improved from 0.74517 to 0.87066, saving model to .\resnet.h5
Epoch 5/20
17/17 [==============================] - 9s 504ms/step - loss: 0.0169 - accuracy: 0.9952 - val_loss: 161.5173 - val_accuracy: 0.1313

Epoch 00005: val_accuracy did not improve from 0.87066
Epoch 6/20
17/17 [==============================] - 8s 483ms/step - loss: 0.0184 - accuracy: 0.9957 - val_loss: 87.2207 - val_accuracy: 0.

C:\Users\FOCUS\AppData\Local\Temp\ipykernel_22608\4074440202.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of Resnet: 98.652%

### Model 6: Inception

In [17]:
def inception( num_class, epochs,savepath='./inception.h5',history=history_Inception,input_shape=INPUT_SIZE):
    model_fine_tune = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:35]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[35:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
    )

In [18]:
start_time=time.time()
inception(num_class=5,epochs=20)	#binary classificaiton

end_time=time.time()
timelist.append(end_time-start_time)

c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 22s 644ms/step - loss: 0.2028 - accuracy: 0.9229 - val_loss: 1.4018 - val_accuracy: 0.7317

Epoch 00001: val_accuracy improved from -inf to 0.73166, saving model to .\inception.h5
Epoch 2/20
17/17 [==============================] - 5s 277ms/step - loss: 0.0169 - accuracy: 0.9952 - val_loss: 8.0309 - val_accuracy: 0.2413

Epoch 00002: val_accuracy did not improve from 0.73166
Epoch 3/20
17/17 [==============================] - 5s 272ms/step - loss: 0.0462 - accuracy: 0.9942 - val_loss: 5.0730 - val_accuracy: 0.1506

Epoch 00003: val_accuracy did not improve from 0.73166
Epoch 00003: early stopping


C:\Users\FOCUS\AppData\Local\Temp\ipykernel_22608\4074440202.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


Validation accuracy of Inception: 100.0%

### Model 7: InceptionResnet

In [19]:
def inceptionresnet( num_class, epochs,savepath='./inceptionresnet.h5',history=history_InceptionResnet,input_shape=INPUT_SIZE):
    model_fine_tune = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:500]:	#the number of frozen layers for transfer learning, have tuned from 400-550
        layer.trainable = False
    for layer in model_fine_tune.layers[500:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[earlyStopping, saveBestModel, history],
    )

In [20]:
start_time=time.time()
inceptionresnet(num_class=5,epochs=20)	# 5-class classificaiton
end_time=time.time()
timelist.append(end_time-start_time)

c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
c:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
17/17 [==============================] - 18s 505ms/step - loss: 0.2535 - accuracy: 0.9041 - val_loss: 8.9571 - val_accuracy: 0.5830

Epoch 00001: val_accuracy improved from -inf to 0.58301, saving model to .\inceptionresnet.h5
Epoch 2/20
17/17 [==============================] - 6s 324ms/step - loss: 0.0429 - accuracy: 0.9918 - val_loss: 855.8776 - val_accuracy: 0.1255

Epoch 00002: val_accuracy did not improve from 0.58301
Epoch 3/20
17/17 [==============================] - 6s 325ms/step - loss: 0.0144 - accuracy: 0.9961 - val_loss: 1335.4979 - val_accuracy: 0.1255

Epoch 00003: val_accuracy did not improve from 0.58301
Epoch 00003: early stopping


C:\Users\FOCUS\AppData\Local\Temp\ipykernel_22608\4074440202.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df=output_df.append(result_dict,ignore_index=True)


In [23]:
output_df.index=output_index
output_df['Time']=timelist
output_df

,Loss,Accuracy,Val-Loss,Val-Accuracy,Time
CNN,0.317030,0.884875,0.176219,0.918919,56.111356
Xception,0.121406,0.963873,0.051983,1.000000,29.523946
VGG16,0.081342,0.965800,0.020728,1.000000,56.129753
VGG19,0.142718,0.949422,0.009572,1.000000,29.495696
Resnet,0.072001,0.974952,2.478381,0.870656,43.197704
Inception,0.202810,0.922929,1.401780,0.731660,35.801315
InceptionResnet,0.253547,0.904143,8.957052,0.583012,33.867579


Validation accuracy of InceptionResnet: 99.993%

# Hyperparameter Optimization 
Use VGG16 as an example.  

Tuned hyperparameters of CNN: 
1. The number of frozen layers
2. The number of epochs
3. Early stop patience
4. Learning rate
5. Dropout rate

Hyperparameter optimization methods:
1. Random search
2. Bayesian optimization - Tree Parzen Estimator(BO-TPE)

In [25]:
def vgg16( num_class,epochs=20,frozen=15,lr=0.001,patience=2, dropout_rate=0.5,verbose=0, savepath='./VGG16.h5',history=history_this,input_shape=INPUT_SIZE):
    model_fine_tune = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[earlyStopping, saveBestModel, history],
        verbose = verbose
    )
    return hist


In [26]:
def prediction(vgg_model):
#read images from validation folder
    rootdir = './test_224/'
    test_laels = []
    test_images=[]
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
                continue
            test_laels.append(subdir.split('/')[-1])
            test_images.append(os.path.join(subdir, file))

    predict=[]
    length=len(test_images)
    label=validation_generator.class_indices
    label={v: k for k, v in label.items()}
    for i in range(length):
        inputimg=test_images[i]
        test_batch=[]
        thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
        #print(thisimg)
        test_shape=(1,)+thisimg.shape
        thisimg=thisimg.reshape(test_shape)
        vgg_model_batch=vgg_model.predict(thisimg) #use master model to process the input image
        #generate result by model 1
        prob=vgg_model_batch[0,np.argmax(vgg_model_batch,axis=1)[0]]
        res=label[np.argmax(vgg_model_batch,axis=1)[0]]
        predict.append(res)
    acc=accuracy_score(test_laels,predict)
    return acc

In [27]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'lr': abs(float(params['lr'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    frozen=params['frozen']
    epochs=params['epochs']
    patience=params['patience']
    lr=params['lr']
    dropout_rate=params['dropout_rate']

    vgg16(num_class=5, frozen=frozen,epochs=epochs,patience=patience, lr=lr, dropout_rate=dropout_rate)

    acc=prediction(vgg_model=load_model('./VGG16.h5'))

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

In [28]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

accuracy:1.0                                                                                                           
 10%|██████▎                                                        | 1/10 [01:18<11:49, 78.79s/trial, best loss: -1.0]

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '



accuracy:1.0                                                                                                           
 20%|████████████▌                                                  | 2/10 [02:01<07:41, 57.70s/trial, best loss: -1.0]

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '



accuracy:1.0                                                                                                           
 30%|██████████████████▉                                            | 3/10 [03:15<07:36, 65.20s/trial, best loss: -1.0]

...

accuracy:0.888030888030888                                                                                             
 80%|██████████████████████████████████████████████████▍            | 8/10 [08:17<02:05, 62.62s/trial, best loss: -1.0]

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '



accuracy:1.0                                                                                                           
 90%|████████████████████████████████████████████████████████▋      | 9/10 [09:11<00:59, 59.98s/trial, best loss: -1.0]

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '



accuracy:0.9575289575289575                                                                                            
100%|██████████████████████████████████████████████████████████████| 10/10 [09:55<00:00, 59.53s/trial, best loss: -1.0]
Hyperopt estimated optimum {'dropout_rate': 0.5, 'epochs': 15.0, 'frozen': 15.0, 'lr': 0.002, 'patience': 3.0}
Time: 595.261168718338


In [29]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))


  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '



accuracy:0.9980694980694981                                                                                            
 10%|████▊                                           | 1/10 [01:06<10:00, 66.70s/trial, best loss: -0.9980694980694981]

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '



accuracy:1.0                                                                                                           
 20%|████████████▌                                                  | 2/10 [01:59<07:46, 58.35s/trial, best loss: -1.0]

...

accuracy:1.0                                                                                                           
 80%|██████████████████████████████████████████████████▍            | 8/10 [06:45<01:41, 50.89s/trial, best loss: -1.0]

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '



accuracy:1.0                                                                                                           
 90%|████████████████████████████████████████████████████████▋      | 9/10 [07:28<00:48, 48.43s/trial, best loss: -1.0]

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(

C:\Users\FOCUS\.conda\envs\tf2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '



accuracy:1.0                                                                                                           
100%|██████████████████████████████████████████████████████████████| 10/10 [08:11<00:00, 49.15s/trial, best loss: -1.0]
Hyperopt estimated optimum {'dropout_rate': 0.5, 'epochs': 15.0, 'frozen': 18.0, 'lr': 0.002, 'patience': 3.0}
Time: 491.4807291030884


In [31]:
# Retrain the model by using the best hyperparameter values to obtain the best model
vgg16(num_class=5, frozen=18,epochs=15,patience=3, lr=0.002, dropout_rate=0.5,verbose=1)

Epoch 1/15
17/17 [==============================] - 5s 266ms/step - loss: 1.0856 - accuracy: 0.6652 - val_loss: 0.6871 - val_accuracy: 0.7336

Epoch 00001: val_accuracy improved from -inf to 0.73359, saving model to .\VGG16.h5
Epoch 2/15
17/17 [==============================] - 4s 251ms/step - loss: 0.5557 - accuracy: 0.8107 - val_loss: 0.3996 - val_accuracy: 0.8610

Epoch 00002: val_accuracy improved from 0.73359 to 0.86100, saving model to .\VGG16.h5
Epoch 3/15
17/17 [==============================] - 4s 256ms/step - loss: 0.3346 - accuracy: 0.8911 - val_loss: 0.2401 - val_accuracy: 0.9788

Epoch 00003: val_accuracy improved from 0.86100 to 0.97876, saving model to .\VGG16.h5
Epoch 4/15
17/17 [==============================] - 4s 257ms/step - loss: 0.2135 - accuracy: 0.9595 - val_loss: 0.1484 - val_accuracy: 0.9884

Epoch 00004: val_accuracy improved from 0.97876 to 0.98842, saving model to .\VGG16.h5
Epoch 5/15
17/17 [==============================] - 4s 252ms/step - loss: 0.1429 - 